In [1]:
import requests # 웹 페이지 소스를 얻기 위한 패키지(기본 내장 패키지이다.)
from bs4 import BeautifulSoup # 웹 페이지 소스를 얻기 위한 패키지, 더 간단히 얻을 수 있다는 장점이 있다고 한다.
from datetime import datetime                                # (!pip install beautifulsoup4 으로 다운받을 수 있다.)
import pandas as pd # 데이터를 처리하기 위한 가장 기본적인 패키지
import time # 사이트를 불러올 때, 작업 지연시간을 지정해주기 위한 패키지이다. (사이트가 늦게 켜지면 에러가 발생하기 때문)
import urllib.request #
from selenium.webdriver import Chrome
import json
import re
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import datetime as dt
import os
from selenium import webdriver
from selenium.webdriver.common.by import By

In [2]:
# 유튜버마다 바꿔줘야 함
start_url = 'https://www.youtube.com/channel/UC-i2ywiuvjvpTy2zW-tXfkw/videos'
youtube_name = input("유튜브 채널 이름을 입력해주세요 : ")
delay=4
browser = Chrome()
browser.implicitly_wait(delay)
delay=4
browser.get(start_url)
browser.maximize_window()

#동영상 클릭
browser.find_element_by_xpath('//*[@id="tabsContent"]/paper-tab[2]').click()

body = browser.find_element_by_tag_name('body')#스크롤하기 위해 소스 추출

num_of_pagedowns = 1
##10번 밑으로 내리는 것
while num_of_pagedowns:
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(2)
    num_of_pagedowns -= 1

html0 = browser.page_source
html = BeautifulSoup(html0,'html.parser')

video_list0 = html.find('div',{'id':'items', 'class':'style-scope ytd-grid-renderer'})
video_list2 = video_list0.find_all('ytd-grid-video-renderer',{'class':'style-scope ytd-grid-renderer'})

base_url = 'http://www.youtube.com'

# 영상별 url 구하기

video_url = []
for i in range(len(video_list2)):
    url = base_url+video_list2[i].find('a',{'id':'thumbnail'})['href']
    video_url.append(url)


video_pd = pd.DataFrame({'name': [],
                         'thumbnail': [],
                         'view': [],
                         'previous_time': [],
                         'video_url': [],
                         'start_date': [],
                         'comment': [],
                         'likes_num': [],
                         'unlikes_num': [],
                         'time_duration': []})
base_url = 'http://www.youtube.com'
delay = 3
browser = Chrome()
browser.implicitly_wait(delay)
browser.maximize_window()

for i in range(0, 20):
    name = video_list2[i].find('a', {'id': 'video-title'}).text

    thum = video_list2[i].find('a', {'id': 'thumbnail'}).find('img')['src']

    url = base_url + video_list2[i].find('a', {'id': 'thumbnail'})['href']

    meta0 = video_list2[i].find('div', {'id': 'metadata-line'})

    view = meta0.find_all('span', {'class': 'style-scope ytd-grid-video-renderer'})[0].text.split()[1]
    # 조회수는 숫자만 뽑을 수 있고 모든 데이터는 원하는 형식으로 뽑을 수 있음
    previous = meta0.find_all('span', {'class': 'style-scope ytd-grid-video-renderer'})[1].text
    start_date = html.find('div', {'id': 'date'}).find('yt-formatted-string').text
    time_duration = video_list2[i].find('span', {'class': 'ytd-thumbnail-overlay-time-status-renderer'}).text

    start_url = video_url[i]
    browser.get(start_url)
    body = browser.find_element_by_tag_name('body')

    time.sleep(1.5)  # 브라우저 로딩시간기다려야함

    num_of_pagedowns = 1  # page_down은 2로 놔도 괜찮을 듯 최대화 했으므로
    while num_of_pagedowns:
        body.send_keys(Keys.PAGE_DOWN)
        time.sleep(1.5)  # 스크롤 속도를 지정할 수 있는 것인데  1.5초가 가장 적당할 듯 1초면 너무 빠름
        # 현재는 스크롤을 7번만 하도록 설정되어 있으나 이후에 댓글을 크롤링하기 위해서는 스크롤을 댓글이 안보
        # 일때까지 내리면 될텐데 시간이 엄청 오래 걸릴 것임 - 따라서 적당한 수준의 댓글을 크롤링하는것이 좋음
        num_of_pagedowns -= 1

    # 여기에 time.sleep를 집어 넣는 방법을 고안
    html0 = browser.page_source
    html = BeautifulSoup(html0, 'html.parser')

    # 자꾸 오류나서 주석처리 ㅠㅠ
    # 댓글 인기순/작성순 선택할 수 있는 영역 클릭
    #     browser.find_element_by_xpath('//paper-button[@class="dropdown-trigger style-scope yt-dropdown-menu"]').click()

    # 댓글 인기순 카테고리 클릭
    #     browser.find_element_by_xpath('//paper-listbox[@class="dropdown-content style-scope yt-dropdown-menu"]/a[1]').click()

    comment = html.find('h2', {'id': 'count'}).find('yt-formatted-string').text

    # 좋아요수
    likes_num = html.find('yt-formatted-string',
                          {'id': 'text', 'class': 'style-scope ytd-toggle-button-renderer style-text',
                           'aria-label': re.compile('좋아요')}).text + '개'

    # 싫어요수
    unlikes_num = html.find('yt-formatted-string',
                            {'id': 'text', 'class': 'style-scope ytd-toggle-button-renderer style-text',
                             'aria-label': re.compile('싫어요')}).text + '개'

    insert_data = pd.DataFrame({'name': [name],
                                'thumbnail': [thum],
                                'view': [view],
                                'previous_time': [previous],
                                'video_url': [url],
                                'start_date': [start_date],
                                'comment': [comment],
                                'likes_num': [likes_num],
                                'unlikes_num': [unlikes_num],
                                'time_duration': [time_duration]})

    video_pd = video_pd.append(insert_data)

    num_page_down = 30
    while num_page_down:
        body.send_keys(Keys.PAGE_DOWN)
        time.sleep(1.5)
        num_page_down -= 1

    comment_pd = pd.DataFrame({'youtube_id': [],
                               'comment': [],
                               'like_num': []})

    html_s0 = browser.page_source
    html_s = BeautifulSoup(html_s0, 'html.parser')

    comment0 = html_s.find_all('ytd-comment-renderer', {'class': 'style-scope ytd-comment-thread-renderer'})

    for j in range(len(comment0)):
        # 댓글
        comment = comment0[j].find('yt-formatted-string',
                                   {'id': 'content-text', 'class': 'style-scope ytd-comment-renderer'}).text

        try:
            aa = comment0[j].find('span', {'id': 'vote-count-left'}).text
            # 정규표현식으로 숫자만 추출하는 것은 정규표현식에 대한 공부를 더 한 뒤 해결
            # re.findall('[0-9]',aa)
            # "".join(re.findall('[0-9]',aa)) -> 리스트 내부의 문자열의 합
            like_num = "".join(re.findall('[0-9]', aa)) + "개"
        except:
            like_num = 0

        bb = comment0[j].find('a', {'id': 'author-text'}).find('span').text
        youtube_id = "".join(re.findall('[가-힣0-9a-zA-Z]', bb))

        insert_data = pd.DataFrame({'youtube_id': [youtube_id],
                                    'comment': [comment],
                                    'like_num': [like_num]})

        comment_pd = comment_pd.append(insert_data)
        comment_pd.index = range(len(comment_pd))

    if not os.path.exists(f'{youtube_name}/comment'):
        os.makedirs(f'{youtube_name}/comment')
    comment_pd.to_csv(f"{youtube_name}/comment/{youtube_name}_comment_{i + 1}.csv", mode='w', encoding='utf-8-sig')

video_pd.index = range(len(video_pd))
if not os.path.exists(f'{youtube_name}'):
    os.makedirs(f'{youtube_name}')
video_pd.to_csv(f"{youtube_name}/{youtube_name}_info.csv", mode='w', encoding='utf-8-sig')

유튜브 채널 이름을 입력해주세요 : 떵개떵
